In [6]:
import os
audio_folder = "C:\\politechnika OKNO\\R4_1PS\\Metody Sztucznej Inteligencji\\projekt\\data"
path = 'C:\\politechnika OKNO\\R4_1PS\\Metody Sztuczej Inteligencji\\projekt\\data'
print(os.listdir(path))

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24', 'archive.zip', 'audio_speech_actors_01-24']


In [9]:
import os
import librosa
import numpy as np
import pandas as pd

# Base folder containing actor folders
audio_folder = r'C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data'
n_mfcc = 20
hop_length = 512

# List of actor folders
actors = ['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 
          'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 
          'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 
          'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']

# Process each actor folder
for actor in actors:
    features_list = []  # Initialize list for each actor's features
    
    actor_folder = os.path.join(audio_folder, actor)
    
    # Ensure actor folder exists to avoid errors
    if not os.path.exists(actor_folder):
        print(f"Folder for {actor} does not exist. Skipping...")
        continue
    
    # Process each .wav file in the actor's folder
    for filename in os.listdir(actor_folder):
        if filename.endswith(".wav"):
            file_path = os.path.join(actor_folder, filename)
            y, sr = librosa.load(file_path, sr=None)
            
            # Extract MFCC and delta MFCC features
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)
            delta_mfccs = librosa.feature.delta(mfccs)
            
            # Calculate mean and variance for MFCCs and delta MFCCs
            mfcc_means = np.mean(mfccs, axis=1)
            mfcc_vars = np.var(mfccs, axis=1)
            delta_mfcc_means = np.mean(delta_mfccs, axis=1)
            delta_mfcc_vars = np.var(delta_mfccs, axis=1)
            
            # Concatenate all features into a single vector
            feature_vector = np.concatenate([mfcc_means, mfcc_vars, delta_mfcc_means, delta_mfcc_vars])
            
            # Append to list with filename
            features_list.append({
                'filename': filename,
                'features': feature_vector
            })
    
    # Convert the list to a DataFrame for this actor
    features_df = pd.DataFrame(features_list)
    
    # Create a folder to save the actor's features if it doesn't exist
    output_folder = os.path.join(audio_folder, 'features_per_actor')
    os.makedirs(output_folder, exist_ok=True)
    
    # Save to CSV in the output folder
    output_path = os.path.join(output_folder, f'{actor}_features.csv')
    features_df.to_csv(output_path, index=False)
    
    print(f"Features for {actor} saved to {output_path}")

print("All features saved successfully!")


Features for Actor_01 saved to C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data\features_per_actor\Actor_01_features.csv
Features for Actor_02 saved to C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data\features_per_actor\Actor_02_features.csv
Features for Actor_03 saved to C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data\features_per_actor\Actor_03_features.csv
Features for Actor_04 saved to C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data\features_per_actor\Actor_04_features.csv
Features for Actor_05 saved to C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data\features_per_actor\Actor_05_features.csv
Features for Actor_06 saved to C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data\features_per_actor\Actor_06_features.csv
Features for Actor_07 saved to C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data\features_per_actor\Actor_07_features.csv
Featur

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# Path to the folder where individual actor feature files are saved
features_folder = r'C:\politechnika OKNO\R4_1PS\Metody Sztuczej Inteligencji\projekt\data\features_per_actor'

# List of actor files
actors = ['Actor_01_features.csv', 'Actor_02_features.csv', 'Actor_03_features.csv', 'Actor_04_features.csv',
          'Actor_05_features.csv', 'Actor_06_features.csv', 'Actor_07_features.csv', 'Actor_08_features.csv',
          'Actor_09_features.csv', 'Actor_10_features.csv', 'Actor_11_features.csv', 'Actor_12_features.csv',
          'Actor_13_features.csv', 'Actor_14_features.csv', 'Actor_15_features.csv', 'Actor_16_features.csv',
          'Actor_17_features.csv', 'Actor_18_features.csv', 'Actor_19_features.csv', 'Actor_20_features.csv',
          'Actor_21_features.csv', 'Actor_22_features.csv', 'Actor_23_features.csv', 'Actor_24_features.csv']

# Initialize lists to collect features and labels
X = []  # Features
y = []  # Labels

# Load data for each actor and assign a unique label
for label, actor_file in enumerate(actors):
    actor_path = os.path.join(features_folder, actor_file)
    actor_data = pd.read_csv(actor_path)
    
    # Assuming the 'features' column contains feature vectors as strings, convert them to numpy arrays
    for _, row in actor_data.iterrows():
        features = np.fromstring(row['features'].strip("[]"), sep=' ')  # Convert string to array
        X.append(features)
        y.append(label)  # Assign a label per actor
    
# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)


In [11]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Initialize and train the Gaussian Naive Bayes classifier
gnb = GaussianNB()
gnb.fit(X_train, y_train)


GaussianNB()

In [13]:
# Make predictions
y_pred = gnb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 77.43%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.64      0.78        11
           1       0.50      0.92      0.65        13
           2       0.75      1.00      0.86         6
           3       1.00      0.58      0.74        12
           4       0.92      0.86      0.89        14
           5       0.23      0.42      0.29        12
           6       1.00      0.93      0.96        14
           7       0.82      1.00      0.90         9
           8       0.90      0.90      0.90        10
           9       0.85      0.69      0.76        16
          10       0.88      0.88      0.88        16
          11       1.00      0.70      0.82        10
          12       1.00      0.77      0.87        13
          13       0.70      0.58      0.64        12
          14       1.00      1.00      1.00        12
          15       0.76      0.87      0.81        15
          16       0.89      0.80      0.